old notebook from KITP...

In [ ]:
import pickle

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import astropy.coordinates as coord
from astropy.coordinates.matrix_utilities import rotation_matrix
from astropy.table import Table
from astropy.io import fits
import astropy.units as u
from ezmist import get_one_isochrone
from scipy.ndimage import gaussian_filter
from scipy.interpolate import InterpolatedUnivariateSpline

import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
import gala.mpl_style

from pyia import GaiaData

from coordinates import pal5_c, galcen_frame, pal5_lead_frame, pal5_trail_frame
from coordinates import trail_epts, lead_epts

In [ ]:
t = Table.read('../data/pal5-apw-filtered.fits')
c = coord.SkyCoord(ra=t['ra']*u.deg, dec=t['dec']*u.deg)

c_l = c.transform_to(pal5_lead_frame)
c_t = c.transform_to(pal5_trail_frame)

Xl = np.stack((c_l.phi1.wrap_at(180*u.deg).degree,
               c_l.phi2.degree)).T

Xt = np.stack((c_t.phi1.wrap_at(180*u.deg).degree,
               c_t.phi2.degree)).T

# Select RR Lyrae:

In [ ]:
t = Table.read('/Users/adrian/data/streams/Pal5/pal5_rrl_inside_canonical_footprint.csv')
t.rename_column('ra_2', 'ra')
t.rename_column('dec_2', 'dec')

rrl = GaiaData(t)
rrl = rrl[(rrl.D_kpc > 18) & (rrl.D_kpc < 24)]

In [ ]:
# plt.scatter(coord.Distance(distmod=rrl.DM).kpc,
#             rrl.D_kpc - coord.Distance(distmod=rrl.DM).kpc)
# plt.xlim(5, 40)
# plt.ylim(-10, 10)

In [ ]:
rrl_c = rrl.get_skycoord(distance=rrl.D_kpc*u.kpc)
rrl_c_pal5 = rrl_c.transform_to(gc.Pal5PriceWhelan18)
rrl_c_pal5_ref = gc.reflex_correct(rrl_c_pal5)
rrl_c_l = rrl_c.transform_to(pal5_lead_frame)
rrl_c_t = rrl_c.transform_to(pal5_trail_frame)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.scatter(rrl_c_pal5.pm_phi1_cosphi2,
           rrl_c_pal5.pm_phi2,
           marker='o', alpha=0.5,
           vmin=-20, vmax=20)
ax.set_xlim(-1, 9)
ax.set_ylim(-5, 5)

ax.set_xlabel(r'$\mu_{\phi_1}$')
ax.set_ylabel(r'$\mu_{\phi_2}$')

In [ ]:
for X, _c in zip([Xl, Xt],
                 [rrl_c_l, rrl_c_t]):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))

    ax.plot(X[:, 0], X[:, 1],
            marker='o', ls='none', 
            color='k', alpha=0.25, ms=2)
    
    ax.scatter(_c.phi1.wrap_at(180*u.deg).degree,
               _c.phi2.degree, color='tab:orange', 
               lw=1., edgecolor='#666666', s=50)

    ax.set_xlim(0, 20.)
    ax.set_ylim(-1.5, 1.5)
    # ax.set_aspect('equal')

    ax.set_xlabel(r'$\phi_1$ [deg]')
    ax.set_ylabel(r'$\phi_2$ [deg]')

    fig.tight_layout()
    fig.set_facecolor('w')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.scatter(rrl_c_pal5.pm_phi1_cosphi2,
           rrl_c_pal5.pm_phi2,
# ax.scatter(rrl_c_pal5_ref.pm_phi1_cosphi2,
#            rrl_c_pal5_ref.pm_phi2,
           marker='o', 
           vmin=-20, vmax=20)
ax.set_xlim(1, 7)
ax.set_ylim(-3, 3)

ax.set_xlabel(r'$\mu_{\phi_1}$')
ax.set_ylabel(r'$\mu_{\phi_2}$')

pm_mask = ((rrl_c_pal5.pm_phi1_cosphi2 > 3*u.mas/u.yr) & 
           (rrl_c_pal5.pm_phi1_cosphi2 < 4.5*u.mas/u.yr) & 
           (rrl_c_pal5.pm_phi2 > 0*u.mas/u.yr) & 
           (rrl_c_pal5.pm_phi2 < 1.2*u.mas/u.yr))

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 8), 
                         sharex=True)

ax = axes[0]
ax.plot(rrl_c_pal5.phi1.wrap_at(180*u.deg).degree[pm_mask],
        rrl_c.distance[pm_mask], 
        marker='o', ls='none', alpha=0.5, color='k')
ax.set_xlim(-25, 25)
# ax.set_ylim(10, 35)
# ax.set_xlim(-10, 10)
ax.set_ylim(17, 25)

ax = axes[1]
ax.plot(rrl_c_pal5.phi1.wrap_at(180*u.deg).degree[pm_mask],
        rrl_c_pal5.pm_phi1_cosphi2[pm_mask], 
        marker='o', ls='none', alpha=0.5, color='k')

ax = axes[2]
ax.plot(rrl_c_pal5.phi1.wrap_at(180*u.deg).degree[pm_mask],
        rrl_c_pal5.pm_phi2[pm_mask], 
        marker='o', ls='none', alpha=0.5, color='k')

ax.xaxis.set_ticks(np.arange(-25, 25+1e-3, 5));

In [ ]:
for X, _c in zip([Xl, Xt],
                 [rrl_c_l, rrl_c_t]):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))

    ax.plot(X[:, 0], X[:, 1],
            marker='o', ls='none', 
            color='k', alpha=0.25, ms=2)
    
    ax.scatter(_c.phi1.wrap_at(180*u.deg).degree[pm_mask],
               _c.phi2.degree[pm_mask], 
               color='tab:orange', zorder=10,
               lw=1., edgecolor='#666666', s=50)

    ax.set_xlim(0, 25.)
    ax.set_ylim(-2.5, 2.5)
    # ax.set_aspect('equal')

    ax.set_xlabel(r'$\phi_1$ [deg]')
    ax.set_ylabel(r'$\phi_2$ [deg]')

    fig.tight_layout()
    fig.set_facecolor('w')

In [ ]:
d_interp = InterpolatedUnivariateSpline([-22., -5, 0, 5],
                                        [23., 21, 20.5, 19.5],
                                        bbox=[-25, 25])

---

## Load photometry and shit

In [ ]:
a2ebv = [3.995, 3.214, 2.165, 1.592, 1.211, 1.064]
t = Table.read('../data/pal5_ls_lite.fits')
t = t[(22.5 - 2.5*np.log10(t['flux_g']) - t['ebv']*a2ebv[1]) < 23.5] # de-reddened g cut
c = coord.SkyCoord(ra=t['ra']*u.deg, dec=t['dec']*u.deg)

In [ ]:
c_pal5 = c.transform_to(gc.Pal5PriceWhelan18)
phi1 = c_pal5.phi1.wrap_at(180*u.deg)

In [ ]:
g0 = 22.5 - 2.5*np.log10(t['flux_g']) - t['ebv']*a2ebv[1]
r0 = 22.5 - 2.5*np.log10(t['flux_r']) - t['ebv']*a2ebv[2]

### Shift CMD by distance

In [ ]:
coord.Distance(d_interp(-20)*u.kpc).distmod, coord.Distance(d_interp(10)*u.kpc).distmod

In [ ]:
M_g = g0 - coord.Distance(d_interp(-20)*u.kpc).distmod.value

## Stellar pop along leading, trailing arms:

In [ ]:
pal5_lead = Table.read('../data/pal5_lead_samples.fits')
pal5_trail = Table.read('../data/pal5_trail_samples.fits')

In [ ]:
lead_c = c.transform_to(pal5_lead_frame)
trail_c = c.transform_to(pal5_trail_frame)

In [ ]:
stream_mask = np.zeros(len(c), dtype=bool)
control_mask = np.zeros(len(c), dtype=bool)
for cc, tbl, name in zip([lead_c, trail_c],
                         [pal5_lead, pal5_trail],
                         ['lead', 'trail']):
    with open('ctl_paths_{}.pkl'.format(name), 'rb') as _f:
        ctl_paths = pickle.load(_f)

    with open('str_path_{}.pkl'.format(name), 'rb') as _f:
        str_path = pickle.load(_f)
        
    X = np.stack((cc.phi1.degree, 
                  cc.phi2.degree)).T
    stream_mask |= str_path.contains_points(X)
    control_mask |= ctl_paths[0].contains_points(X) | ctl_paths[1].contains_points(X)

In [ ]:
iso = Table.read('/Users/adrian/data/Isochrones/MIST/FeH_-1.3_iso.fits')
iso1 = iso[iso['log10_isochrone_age_yr'] == 10.1]
phasecut = (iso1['phase'] >= 0) & (iso1['phase'] < 4)
iso1 = iso1[phasecut]

iso_g = iso1['dec_g']
iso_r = iso1['dec_r']

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6), 
                         sharex=True, sharey=True)

cmd_bins = (np.arange(-0.5, 1.1+1e-3, 0.02),
            np.arange(0, 7+1e-3, 0.04))

cl_mask = c.separation(pal5_c) < 0.15*u.deg

tail_mask = np.logical_not(cl_mask)

ax = axes[0]
H1, xe, ye = np.histogram2d((g0-r0)[stream_mask & tail_mask],
                            M_g[stream_mask & tail_mask],
                            bins=cmd_bins)
ax.pcolormesh(xe, ye, H1.T, cmap='magma')

ax = axes[1]
H2, xe, ye = np.histogram2d((g0-r0)[control_mask & tail_mask],
                            M_g[control_mask & tail_mask],
                            bins=cmd_bins)
ax.pcolormesh(xe, ye, H2.T, cmap='magma')

ax = axes[2]
H1 = gaussian_filter(H1, 1)
H2 = gaussian_filter(H2, 1)
ax.pcolormesh(xe, ye, (H1 - H2).T, cmap='Greys', 
              norm=mpl.colors.LogNorm(vmin=0.5, vmax=20))

ax.set_xlim(-0.5, 1.1)
ax.set_ylim(7, 0)

fig.tight_layout()

## Stars in the cluster itself

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(c.ra.deg, c.dec.deg,
        marker='.', ls='none', alpha=0.5)

lim1 = 0.075 * u.deg
lim2 = 0.2 * u.deg 
ax.add_patch(mpl.patches.Circle((pal5_c.ra.deg, pal5_c.dec.deg),
                                radius=lim1.value, facecolor='k', 
                                alpha=0.2, zorder=100))

ax.add_patch(mpl.patches.Circle((pal5_c.ra.deg, pal5_c.dec.deg),
                                radius=lim2.value, facecolor='k', 
                                alpha=0.2, zorder=100))

ax.set_xlim(pal5_c.ra.deg+1, pal5_c.ra.deg-1)
ax.set_ylim(pal5_c.dec.deg-1, pal5_c.dec.deg+1)

In [ ]:
cl_mask = (c.separation(pal5_c) > lim1) & (c.separation(pal5_c) < lim2)
cl_mask.sum()

In [ ]:
A = (lim2**2 - lim1**2)
r1 = np.sqrt(A)
off1 = coord.SkyCoord(228.4*u.deg, 0.25*u.deg)
bg_mask = (c.separation(off1) < r1)
bg_mask.sum()

In [ ]:
iso = Table.read('/Users/adrian/data/Isochrones/MIST/FeH_-1.3_iso.fits')
iso1 = iso[iso['log10_isochrone_age_yr'] == 10.1]
phasecut = (iso1['phase'] >= 0) & (iso1['phase'] < 4)
iso1 = iso1[phasecut]

iso_g = iso1['dec_g']
iso_r = iso1['dec_r']

# ---

iso2 = iso[iso['log10_isochrone_age_yr'] == 9.5]
phasecut = (iso2['phase'] >= 0) & (iso2['phase'] < 4)
iso2 = iso2[phasecut]

iso2_g = iso2['dec_g']
iso2_r = iso2['dec_r']

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5),
                         sharex=True, sharey=True)

ax = axes[0]
ax.plot((g0-r0)[cl_mask], 
        M_g[cl_mask],
        marker='o', ls='none', 
        alpha=0.24, color='k')

ax = axes[1]
ax.plot((g0-r0)[bg_mask], 
        M_g[bg_mask],
        marker='o', ls='none', 
        alpha=0.24, color='k')

ax = axes[2]
bins = (np.arange(-0.5, 1.+1e-3, 0.02),
        np.arange(0, 7+1e-3, 0.05))
H1, xe, ye = np.histogram2d((g0-r0)[cl_mask], g0[cl_mask], bins=bins)
H2, xe, ye = np.histogram2d((g0-r0)[bg_mask], g0[bg_mask], bins=bins)

H1 = gaussian_filter(H1, 1.5)
H2 = gaussian_filter(H2, 1.5)

ax.pcolormesh(xe, ye, (H1 - H2).T,
              cmap='Greys', 
              norm=mpl.colors.LogNorm(vmin=1e-2, vmax=10))

ax = axes[0]
ax.plot(iso_g-iso_r - 0.01*(iso_g-3.5)**1.5, 
        iso_g-0.15)
ax.plot(iso_g-iso_r + 0.1 + 0.01*(iso_g-3.5)**1.5, 
        iso_g-0.15)
poly1 = np.stack((iso_g-iso_r - 0.01*(iso_g-3.5)**1.5, 
                  iso_g-0.15)).T
poly2 = np.stack((iso_g-iso_r + 0.1 + 0.01*(iso_g-3.5)**1.5, 
                  iso_g-0.15)).T
grg_path = mpl.path.Path(np.vstack((poly2[poly2[:, 1]<6.8][::-1],
                                    poly1[poly1[:, 1]<6.8])))
# ax.add_patch(mpl.patches.Polygon(grg_path.vertices))

ax.set_xlim(-0.5, 1.)
ax.set_ylim(7, 0)

fig.tight_layout()

In [ ]:
poly_mask = grg_path.contains_points(np.stack((g0-r0, M_g)).T[cl_mask])
poly_mask_bg = grg_path.contains_points(np.stack((g0-r0, M_g)).T[bg_mask])

poly_mask_tail = grg_path.contains_points(np.stack((g0-r0, M_g)).T[stream_mask & tail_mask])
poly_mask_bg_tail = grg_path.contains_points(np.stack((g0-r0, M_g)).T[control_mask & tail_mask])

In [ ]:
g_bins = np.arange(3, 7+1e-3, 0.5)
N_cl, _ = np.histogram(M_g[cl_mask][poly_mask], g_bins)
N_cl_bg, _ = np.histogram(M_g[bg_mask][poly_mask_bg], g_bins)

N_tail, _ = np.histogram(M_g[stream_mask & tail_mask][poly_mask_tail], g_bins)
N_tail_bg, _ = np.histogram(M_g[control_mask & tail_mask][poly_mask_bg_tail], g_bins)

g_bin_c = 0.5*(g_bins[:-1]+g_bins[1:])

plt.errorbar(g_bin_c, N_cl,
             np.sqrt(N_cl), 
             ls='none', marker='o')

plt.errorbar(g_bin_c, N_cl_bg,
             np.sqrt(N_cl_bg), 
             ls='none', marker='o')

plt.errorbar(g_bin_c, N_cl - N_cl_bg,
             np.sqrt(N_cl - N_cl_bg), 
             ls='none', marker='o', color='k')

In [ ]:
kroupa_ms = np.load('/Users/adrian/Downloads/kroupa_masses.npy')

In [ ]:
def dN_dm_Grillmair(m, a=0.02, b=1.):
    return 2/3*(b**1.5-a**1.5) * m**0.5

In [ ]:
ymax = dN_dm_Grillmair(np.linspace(0.4,1,1024), 0.4, 1.).max()
xs = np.random.uniform(0.4, 1., size=500000)
ys = np.random.uniform(0, ymax, size=500000)
grillmair_ms = xs[ys < dN_dm_Grillmair(xs, 0.4, 1.)]

In [ ]:
x = iso1['star_mass'][iso1['phase'] < 2]
y = iso_g[iso1['phase'] < 2]
interp_m2g = InterpolatedUnivariateSpline(x[np.argsort(x)], 
                                          y[np.argsort(x)],
                                          ext=1)
interp_g2m = InterpolatedUnivariateSpline(y[np.argsort(y)], 
                                          x[np.argsort(y)],
                                          ext=1)

In [ ]:
kroupa_gs = interp_m2g(kroupa_ms[:100000])
grillmair_gs = interp_m2g(grillmair_ms)

In [ ]:
N_kr, _ = np.histogram(kroupa_gs, g_bins)
N_gr, _ = np.histogram(grillmair_gs, g_bins)

fig, axes = plt.subplots(1, 2, figsize=(10, 5),
                         sharex=True)

ax = axes[0]
for ax, Nnorm in zip(axes, [(N_cl - N_cl_bg)[1],
                            (N_tail - N_tail_bg)[1]]):
    ax.plot(g_bin_c, N_kr / N_kr[1] * Nnorm, 
            marker='', drawstyle='steps-mid',
            label='kroupa')
    
    ax.plot(g_bin_c, N_gr / N_gr[1] * Nnorm, 
            marker='', drawstyle='steps-mid',
            label='kroupa')

axes[0].errorbar(g_bin_c, N_cl - N_cl_bg,
             np.sqrt(N_cl - N_cl_bg), 
             ls='none', marker='o',
            color='k',
             label='pal 5 cluster LF')

axes[1].errorbar(g_bin_c, N_tail - N_tail_bg,
             np.sqrt(N_tail - N_tail_bg), 
             ls='none', marker='o', 
             color='tab:red',
             label='pal 5 stream LF')

ax.set_xlim(3, 7)
ax.xaxis.set_ticks(np.arange(3, 7+1e-3, 0.5))

for ax in axes:
    ax.set_xlabel('$g$ [mag]')
axes[0].set_ylabel('$N$')

axes[0].set_title('Cluster')
axes[1].set_title('Stream')

fig.set_facecolor('w')